
# Riyadh Stations — Merge & Clean

In [ ]:

import os, re, pandas as pd, numpy as np

CLEANED_PATH = "cleaned_dataset.csv"
STATIONS_PATH = "Riyadh_Stations_TwoU.csv"

def find_file(keyword, folder="."):
    for f in os.listdir(folder):
        if keyword.lower() in f.lower():
            return os.path.join(folder, f)
    return None

def resolve(p):
    if os.path.exists(p):
        return p
    alt = find_file(os.path.splitext(os.path.basename(p))[0], "/mnt/data")
    if alt:
        return alt
    raise FileNotFoundError(p)

def read_csv_any(path):
    try:
        return pd.read_csv(path, sep=None, engine="python", encoding="utf-8-sig")
    except Exception:
        try:
            return pd.read_csv(path, encoding="utf-8-sig")
        except Exception:
            return pd.read_csv(path, sep=";", encoding="utf-8-sig")

df1 = read_csv_any(resolve(CLEANED_PATH))
df2 = read_csv_any(resolve(STATIONS_PATH))

df1.head(2), df2.head(2)


In [ ]:

def norm(s: str) -> str:
    s = str(s).lower().strip()
    s = re.sub(r"\s+", " ", s)
    s = re.sub(r"[^\w\s]", "", s)
    s = s.replace("defence", "defense")
    s = s.replace("fahd", "fahad")
    return s

df1["_k"] = df1["Name"].map(norm)
df2["_k"] = df2["Station name"].map(norm)

ALIAS = {
    "pnu 1": "pnu1",
    "pnu1": "pnu1",
    "pnu 2": "pnu2",
    "pnu2": "pnu2",
    "princess noura university 1": "pnu1",
    "princess noura university 2": "pnu2",
    "metro station a1": "pnu1",
    "metro station a2": "pnu1",
    "metro station a10": "pnu1",
    "metro station f1": "pnu1",
    "metro station s1": "pnu2",
    "metro station 4": "pnu2",
    "metro station a7": "pnu2",
    "metro station a6": "pnu2",
    "a5": "pnu2",
    "ministry of defence": "ministry of defense",
    "subway": "subway",
    "subway  صب واي": "subway",
    "subway salman al farsi": "subway",
}

apply_alias = lambda k: ALIAS.get(k, k)

df1["_k2"] = df1["_k"].apply(apply_alias)
df2["_k2"] = df2["_k"].apply(apply_alias)

df2u = df2.drop_duplicates(subset=["_k2"], keep="first").copy()

cols_to_add = ["Metro line number", "Metro line name", "Station type"]
df_merged = df1.merge(df2u[["_k2"] + cols_to_add], on="_k2", how="left").drop(columns=["_k","_k2"])

def canonical_name(original: str) -> str:
    k = norm(original)
    k2 = apply_alias(k)
    if k2 == "pnu1":
        return "PNU1"
    if k2 == "pnu2":
        return "PNU2"
    return original

df_merged["CanonicalName"] = df_merged["Name"].apply(canonical_name)

# Consolidate duplicates with weighted ratings
g = df_merged.groupby("CanonicalName", dropna=False)
tmp = g.agg({
    "Type_of_Utility": lambda s: s.dropna().mode().iloc[0] if not s.dropna().mode().empty else s.dropna().iloc[0] if s.dropna().shape[0] else np.nan,
    "Number_of_Ratings": "sum",
    "Rating": "mean",
    "Longitude": "mean",
    "Latitude": "mean",
    "Metro line number": lambda s: s.dropna().iloc[0] if s.dropna().shape[0] else np.nan,
    "Metro line name": lambda s: s.dropna().iloc[0] if s.dropna().shape[0] else np.nan,
    "Station type": lambda s: s.dropna().iloc[0] if s.dropna().shape[0] else np.nan,
}).reset_index()

weighted = (
    df_merged
    .assign(_val=pd.to_numeric(df_merged["Rating"], errors="coerce"),
            _w=pd.to_numeric(df_merged["Number_of_Ratings"], errors="coerce").fillna(0))
    .groupby("CanonicalName")
    .apply(lambda d: (d["_val"].fillna(0) * d["_w"]).sum() / d["_w"].sum() if d["_w"].sum() > 0 else d["_val"].mean())
    .rename("Rating")
    .reset_index()
)

df_final = tmp.drop(columns=["Rating"]).merge(weighted, on="CanonicalName", how="left")

# Drop obvious empty rows
key_numeric = ["Rating", "Number_of_Ratings", "Longitude", "Latitude"]
df_final = df_final[~df_final["CanonicalName"].isna()]
df_final = df_final[~df_final[key_numeric].isna().all(axis=1)].reset_index(drop=True)

df_final.head(10)

In [ ]:

OUT_CSV = "final_dataset_cleaned_v2.csv"
df_final.to_csv(OUT_CSV, index=False)
OUT_CSV, df_final[df_final["CanonicalName"].str.contains("PNU", na=False)][["CanonicalName","Metro line number","Metro line name","Station type","Number_of_Ratings","Rating"]]
